In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
# Download dataset
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
!pwd

--2024-05-26 17:15:39--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.193.24.207, 2600:1417:76:581::317f, 2600:1417:76:585::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.193.24.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   439MB/s    in 1.8s    

2024-05-26 17:15:41 (439 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]

/content


In [ ]:
from typing_extensions import TypeVarTuple
from posix import truncate
import os
import zipfile

# Extract archives
zip_ref = zipfile.ZipFile("./kagglecatsanddogs_5340.zip")
zip_ref.extractall('data/')
zip_ref.close()

# Assign Training and Validation Set
base_dir = "./data/PetImages"
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)

# Training Directories
train_dog_dir = os.path.join(train_dir, 'dog/')
os.makedirs(train_dog_dir, exist_ok=True)
train_cat_dir = os.path.join(train_dir, 'cat/')
os.makedirs(train_cat_dir, exist_ok=True)

# Validation Directory
val_dog_dir = os.path.join(validation_dir, 'dog/')
os.makedirs(val_dog_dir, exist_ok=True)
val_cats_dir = os.path.join(validation_dir, 'cat/')
os.makedirs(val_cats_dir, exist_ok=True)

In [ ]:
import random
import shutil

def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  file_list = os.listdir(SOURCE_DIR)
  random.shuffle(file_list)

  # calculate split index
  split_index = int(len(file_list) * SPLIT_SIZE)
   # copy file to training directory
  for i in range(split_index):
    file = file_list[i]
    source_path = os.path.join(SOURCE_DIR, file)
    if os.path.getsize(source_path) == 0:
      print('filename is zero length, so ignoring.')
    else:
      destination_path = os.path.join(TRAINING_DIR, file)
      shutil.copyfile(source_path, destination_path)

  for i in range(split_index, len(file_list)):
    file = file_list[i]
    source_path = os.path.join(SOURCE_DIR, file)
    if (os.path.getsize(source_path) == 0):
      print("filename is zero length, so ignoring.")
    else:
      destination_path = os.path.join(VALIDATION_DIR, file)
      shutil.copyfile(source_path, destination_path)


In [ ]:
# Define paths
CAT_SOURCE_DIR = "/content/data/PetImages/Cat"
DOG_SOURCE_DIR = "/content/data/PetImages/Dog"

# Empty directories in case you run this cell multiple times
if len(os.listdir(train_cat_dir)) > 0:
  for file in os.scandir(train_cat_dir):
    os.remove(file.path)
if len(os.listdir(train_dog_dir)) > 0:
  for file in os.scandir(train_dog_dir):
    os.remove(file.path)
if len(os.listdir(val_cats_dir)) > 0:
  for file in os.scandir(val_cats_dir):
    os.remove(file.path)
if len(os.listdir(val_dog_dir)) > 0:
  for file in os.scandir(val_dog_dir):
    os.remove(file.path)

# Define proportion of images used for training
split_size = .9

split_data(CAT_SOURCE_DIR, train_cat_dir, val_cats_dir, split_size)
split_data(DOG_SOURCE_DIR, train_dog_dir, val_dog_dir, split_size)

# Check that the number of images matches the expected output

# Your function should perform copies rather than moving images so original directories should contain unchanged images
print(f"\n\nOriginal cat's directory has {len(os.listdir(CAT_SOURCE_DIR))} images")
print(f"Original dog's directory has {len(os.listdir(DOG_SOURCE_DIR))} images\n")

# Training and validation splits
print(f"There are {len(os.listdir(train_cat_dir))} images of cats for training")
print(f"There are {len(os.listdir(train_dog_dir))} images of dogs for training")
print(f"There are {len(os.listdir(val_cats_dir))} images of cats for validation")
print(f"There are {len(os.listdir(val_dog_dir))} images of dogs for validation")

filename is zero length, so ignoring.
filename is zero length, so ignoring.


Original cat's directory has 12501 images
Original dog's directory has 12501 images

There are 11249 images of cats for training
There are 11249 images of dogs for training
There are 1251 images of cats for validation
There are 1251 images of dogs for validation


In [ ]:
# Create Model
def create_model():
  """
  Creates a CNN with 7 convolutional layers
  """

  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), metrics=['accuracy'])

  return model

In [ ]:
# Preprocess image with Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 40,
		width_shift_range=0.2,
		shear_range=0.2,
		zoom_range=0.2,
		horizontal_flip=True,
		fill_mode='nearest'
)

test_data_gen = ImageDataGenerator(rescale=1./255)


training_generator = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = test_data_gen.flow_from_directory(validation_dir, target_size=(150, 150), batch_size=20, class_mode='binary')

model = create_model()

history = model.fit(training_generator, steps_per_epoch=100, epochs=20, validation_data = validation_generator, validation_steps=50, verbose=2)

Found 22496 images belonging to 2 classes.
Found 2502 images belonging to 2 classes.
Epoch 1/20
100/100 - 19s - loss: 0.6973 - accuracy: 0.5275 - val_loss: 0.6885 - val_accuracy: 0.5210 - 19s/epoch - 189ms/step
Epoch 2/20
100/100 - 13s - loss: 0.6917 - accuracy: 0.5505 - val_loss: 0.6727 - val_accuracy: 0.6320 - 13s/epoch - 129ms/step
Epoch 3/20
100/100 - 13s - loss: 0.6663 - accuracy: 0.6035 - val_loss: 0.6314 - val_accuracy: 0.6600 - 13s/epoch - 129ms/step
Epoch 4/20
100/100 - 13s - loss: 0.6568 - accuracy: 0.6050 - val_loss: 0.7782 - val_accuracy: 0.5180 - 13s/epoch - 128ms/step
Epoch 5/20
100/100 - 13s - loss: 0.6455 - accuracy: 0.6235 - val_loss: 0.6031 - val_accuracy: 0.6810 - 13s/epoch - 127ms/step
Epoch 6/20
100/100 - 13s - loss: 0.6404 - accuracy: 0.6425 - val_loss: 0.6586 - val_accuracy: 0.5880 - 13s/epoch - 125ms/step
Epoch 7/20
100/100 - 13s - loss: 0.6272 - accuracy: 0.6530 - val_loss: 0.6058 - val_accuracy: 0.6780 - 13s/epoch - 128ms/step
Epoch 8/20
100/100 - 13s - loss: 

/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


100/100 - 13s - loss: 0.5655 - accuracy: 0.6935 - val_loss: 0.5148 - val_accuracy: 0.7580 - 13s/epoch - 126ms/step
Epoch 16/20
100/100 - 13s - loss: 0.5652 - accuracy: 0.7114 - val_loss: 0.5079 - val_accuracy: 0.7610 - 13s/epoch - 128ms/step
Epoch 17/20
100/100 - 13s - loss: 0.5573 - accuracy: 0.7120 - val_loss: 0.4735 - val_accuracy: 0.7810 - 13s/epoch - 127ms/step
Epoch 18/20
100/100 - 13s - loss: 0.5465 - accuracy: 0.7280 - val_loss: 0.5139 - val_accuracy: 0.7290 - 13s/epoch - 127ms/step
Epoch 19/20
100/100 - 13s - loss: 0.5512 - accuracy: 0.7265 - val_loss: 0.4555 - val_accuracy: 0.7910 - 13s/epoch - 127ms/step
Epoch 20/20
100/100 - 13s - loss: 0.5454 - accuracy: 0.7245 - val_loss: 0.4958 - val_accuracy: 0.7620 - 13s/epoch - 129ms/step


In [ ]:
import numpy as np

from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded=files.upload()

for fn in uploaded.keys():

  # predicting images
  path='/content/' + fn
  img=load_img(path, target_size=(150, 150))

  x=img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)

  print(classes[0])

  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")

Saving Cat03.jpg to Cat03.jpg
1/1 [==============================] - 0s 17ms/step
[0.13900717]
Cat03.jpg is a cat


In [ ]:
model.save('model.keras')